In [1]:
# Import dependencies
from splinter import Browser
from bs4 import BeautifulSoup as soup
import pandas as pd
import time

In [4]:
# Set up Splinter
# Make sure chromedriver.exe is in PATH
browser= Browser('chrome')
# Store url and visit
base_url='https://www.cars.com/shopping/results/?makes[]=honda&maximum_distance=150&models[]=honda-pilot&page={}&page_size=100&stock_type=all&zip=61606'
page_number = 1
url = base_url.format(page_number)

# Create lists to store details
models = []
statuses = []
mileages = []
prices = []
dealers = []
cities = []
urls = []  # List to store URLs

# Set to store unique identifiers of scraped cars
scraped_car_ids = set()

# Function to scrape current page
def scrape_page(url):
    browser.visit(url)
    time.sleep(5)  # Adjust as necessary
    
    html = browser.html
    car_soup = soup(html, 'html.parser')
    car_list = car_soup.find_all('div', class_='vehicle-card')

    if not car_list:
        print("No more car listings found. Exiting.")
        return False

    for car in car_list:
        # Extract car model
        car_model = car.find('h2', class_='title')
        model = str(car_model.get_text(strip=True)) if car_model else 'N/A'

        # Extract car mileage
        car_mileage = car.find('div', class_='mileage')
        mileage = str(car_mileage.get_text(strip=True)) if car_mileage else 'N/A'

        # Extract car status
        car_status = car.find('p', class_='stock-type')
        status = str(car_status.get_text(strip=True)) if car_status else 'N/A'

        # Extract car price
        car_price = car.find('span', class_='primary-price')
        price = str(car_price.get_text(strip=True)) if car_price else 'N/A'

        # Extract dealer name
        car_dealer_div = car.find('div', class_='dealer-name')
        if car_dealer_div:
            car_dealer = car_dealer_div.find('strong')
            dealer = str(car_dealer.get_text(strip=True)) if car_dealer else 'N/A'
        else:
            dealer = 'N/A'

        # Extract dealer city
        dealer_city = car.find('div', class_='miles-from')
        city = str(dealer_city.get_text(strip=True)) if dealer_city else 'N/A'

        # Extract car URL
        car_url = car.find('a', class_='vehicle-card-link')['href'] if car.find('a', class_='vehicle-card-link') else 'N/A'

        # Check if this car has already been scraped
        if car_url in scraped_car_ids:
            continue  # Skip if already scraped
        
        # Add car URL to set of scraped car IDs
        scraped_car_ids.add(car_url)

        # Append details to lists
        models.append(model)
        statuses.append(status)
        prices.append(price)
        dealers.append(dealer)
        cities.append(city)
        mileages.append(mileage)
        urls.append(car_url)  # Append URL to list

    return True

# Counter to track consecutive pages with no new data
no_new_data_count = 0
max_no_new_data_allowed = 3  # Adjust as necessary

# Timeout settings
timeout_seconds = 240  # 4 minutes timeout
start_time = time.time()

# Loop through pages
while True:
    if time.time() - start_time > timeout_seconds:
        print(f"Timeout ({timeout_seconds} seconds) exceeded. Exiting.")
        break
    
    if not scrape_page(url):
        # Increment no_new_data_count if no new data found
        no_new_data_count += 1
        if no_new_data_count >= max_no_new_data_allowed:
            print(f"No new data found for {max_no_new_data_allowed} consecutive pages. Exiting.")
            break
    else:
        # Reset no_new_data_count if new data found
        no_new_data_count = 0
    
    # Increment page number and update URL
    page_number += 1
    url = base_url.format(page_number)
    print(f"Scraping page {page_number}...")

# Print number of cars scraped
print(f"Number of unique cars scraped: {len(scraped_car_ids)}")

Scraping page 2...
Scraping page 3...
Scraping page 4...
Scraping page 5...
Scraping page 6...
Scraping page 7...
Scraping page 8...
Scraping page 9...
Scraping page 10...
Scraping page 11...
Scraping page 12...
Scraping page 13...
Scraping page 14...
Scraping page 15...
Scraping page 16...
Scraping page 17...
Scraping page 18...
Scraping page 19...
Scraping page 20...
Scraping page 21...
Scraping page 22...
Scraping page 23...
Scraping page 24...
Scraping page 25...
Scraping page 26...
Scraping page 27...
Scraping page 28...
Scraping page 29...
Scraping page 30...
Scraping page 31...
Scraping page 32...
Scraping page 33...
Scraping page 34...
Scraping page 35...
Scraping page 36...
Timeout (240 seconds) exceeded. Exiting.
Number of unique cars scraped: 1026


In [5]:
# Create df
data={
    'Model': models,
    'Mileage (mi)': mileages,
    'Status': statuses,
    'Price (USD)': prices,
    'Dealer': dealers,
    'City': cities ,
    'URL': urls
    }
car_df= pd.DataFrame(data)

In [6]:
# Close the browser
browser.quit()

In [7]:
# Display DF
print("Original DataFrame:")
print(car_df)
print()

Original DataFrame:
                                   Model Mileage (mi)           Status  \
0                  2021 Honda Pilot EX-L   23,048 mi.             Used   
1                  2016 Honda Pilot EX-L  149,646 mi.             Used   
2                  2020 Honda Pilot EX-L   50,994 mi.             Used   
3     2025 Honda Pilot AWD Black Edition          N/A              New   
4                 2024 Honda Pilot Elite    6,536 mi.             Used   
...                                  ...          ...              ...   
1021    2022 Honda Pilot Special Edition   46,064 mi.  Honda Certified   
1022              2023 Honda Pilot Elite   19,892 mi.  Honda Certified   
1023               2020 Honda Pilot EX-L   74,519 mi.             Used   
1024              2023 Honda Pilot Elite   11,740 mi.             Used   
1025            2016 Honda Pilot Touring  108,486 mi.             Used   

     Price (USD)                                             Dealer  \
0        $30,997    

In [8]:
# Check for missing values:
missing_values= car_df.isnull().sum()
print("Missing values:")
print(missing_values)
print()

Missing values:
Model           0
Mileage (mi)    0
Status          0
Price (USD)     0
Dealer          0
City            0
URL             0
dtype: int64



In [9]:
# Split 'Model' column into 'Year', 'Manufacturer', 'Model', 'Trim'
car_df[['Year', 'Manufacturer', 'Model', 'Trim']] = car_df['Model'].str.split(' ', 3, expand=True)

# Replace 'Not Priced' with NaN and convert 'Price' column to numeric
car_df['Price (USD)'] = pd.to_numeric(car_df['Price (USD)'].replace('Not Priced', pd.NA).str.replace('[\$,]', '', regex=True), errors='coerce')

# Clean Mileage column
car_df['Mileage (mi)'] = pd.to_numeric(car_df['Mileage (mi)'].replace('N/A', pd.NA).str.replace('mi.', '').str.replace(',', '').str.strip(), errors='coerce')

# Reorder columns as required
car_df = car_df[['Year', 'Manufacturer', 'Model', 'Trim', 'Mileage (mi)', 'Status', 'Price (USD)', 'Dealer', 'City', 'URL']]

# Display the updated DataFrame
print("Final DataFrame:")
print(car_df)

Final DataFrame:
      Year Manufacturer  Model               Trim  Mileage (mi)  \
0     2021        Honda  Pilot               EX-L       23048.0   
1     2016        Honda  Pilot               EX-L      149646.0   
2     2020        Honda  Pilot               EX-L       50994.0   
3     2025        Honda  Pilot  AWD Black Edition           NaN   
4     2024        Honda  Pilot              Elite        6536.0   
...    ...          ...    ...                ...           ...   
1021  2022        Honda  Pilot    Special Edition       46064.0   
1022  2023        Honda  Pilot              Elite       19892.0   
1023  2020        Honda  Pilot               EX-L       74519.0   
1024  2023        Honda  Pilot              Elite       11740.0   
1025  2016        Honda  Pilot            Touring      108486.0   

               Status  Price (USD)  \
0                Used      30997.0   
1                Used      15495.0   
2                Used      28299.0   
3                 New     

C:\Users\awsom\AppData\Local\Temp\ipykernel_7112\2381806475.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  car_df['Mileage (mi)'] = pd.to_numeric(car_df['Mileage (mi)'].replace('N/A', pd.NA).str.replace('mi.', '').str.replace(',', '').str.strip(), errors='coerce')


In [10]:
car_df.to_excel("Pilot.xlsx")